# Práctica 5 - Isac Añor Santana

Reconocimiento de matrículas.

## Paquetes necesarios
Adicionalmente, es necesario agregar los siguiente paquetes al entorno virtual:

`pip install imutils`

`pip install scikit-image`

In [1]:
import os
import cv2
import glob
import imutils
import pytesseract
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import clear_border

## Definición de la clase de reconocimiento de matrículas y funciones auxiliares


### Clase ALPR (Automatic License Plate Recognition)
Esta clase se encarga del reconocimiento de la matrícula y la extracción de su texto usando el motor de Tesseract.

El reconocimiento de la matrícula se hace en dos fases:
1. En la primera fase se buscan candidatos a ser matrículas con el siguiente algoritmo:
    - En primer lugar, se hace una transformación morfológica de "sombrero negro" o Black Hat, resultado de la diferencia entre el cerrado de la imagen y la imagen original. La finalidad simplificar la detección asumiendo que las matrículas generalmente están constituidas por un fondo claro y un primer plano oscuro (los caracteres). La aplicacón de esta transformación morfológica revelará los caracteres negros sobre un fondo claro.
    - En segundo lugar, se aplica una operación de cerrado para llenar pequeños agujeros e identificar estructuras más grandes en la imagen. Estas estructuras se usarán más adelante.
    - Se aplica el gradiente de Scharr a la imagen resultante de la transformación morforlógica de Black Hat, con la finalidad de detectar los bordes en la imagen y especialmente remarcar los límites de los caracteres de la matrícula.
    - Se prosigue con un desenfoque Gaussiano para suavizar la imagen y agrupar las regiones que puedan contener los límites de los caracteres de la matrícula.
    - Se realiza una operación de erosion y dilatación para reducir el ruido.
    - Finalmente se realiza una comparación lógica de AND bit a bit usando la imagen de estructuras como máscara, seguido de una dilatación y una erosión para reducir el ruido.
    - De esta imagen resultante se extraen los contornos de mayor tamaño.

2. En la segunda fase, de entre los candidatos, se hace un filtrado por relación de aspecto y se devuelve tanto el recorte de la imagen como el contorno. Se asume que aquellos contornos con la relación de aspecto propuesta son matrículas.

Finalmente, los contornos seleccionados se pasan por el "reconocedor óptico de caracteres" Tesseract. Si reconoce algo, se muestra el contorno en la imagen con lo que ha reconocido.


Entre varias fuentes de internet, se destaca el uso de:

https://pyimagesearch.com/2020/09/21/opencv-automatic-license-number-plate-recognition-anpr-with-python/

https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html 

https://iopscience.iop.org/article/10.1088/1742-6596/806/1/012004/pdf 

In [2]:
class AutomaticLicensePlateRecognition:
    def __init__(self, minAspectRatio = 4, maxAspectRatio = 6, debug = False):
        """
        Will initialize the minimum and maximum aspect ratios.
        Also whether or not debug mode is present.

        :param int minAspectRatio: Minimum aspect ratio
        :param int maxAspectRatio: Maximum aspect ratio
        :param bool debug: Whether or not in debug mode
        """
        self.minAspectRatio = minAspectRatio
        self.maxAspectRatio = maxAspectRatio
        self.debug = debug

    def concatenate_horizontally(self, image1, image2):
        """
        Will concatenate two images horizontally

        :param image image1: Image 1
        :param image image2: Image 2
        """
        return np.concatenate((image1, image2), axis = 1)

    def concatenate_vertically(self, image1, image2):
        """
        Will concatenate two images vertically

        :param image image1: Image 1
        :param image image2: Image 2
        """
        return np.concatenate((image1, image2), axis = 0)
    
    def debug_imshow(self, title, img, waitKey = False):
        """
        Will show the image in case of being in debug mode.

        :param str title: Title of the frame created by imshow
        :param image img: Image to display
        :param bool waitKey: Flag to see if the display should wait for a keyboard press
        """
        if self.debug:
            cv2.imshow(title, img)
            # check to see if we should wait for a keypress
            if waitKey:
                cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    def locate_license_plate_candidates(self, gray, keep=40):
        """
        Wilkl perform a blackhat morphological operation that will reveal dark regions (i.e., text) on light backgrounds.
        Morphological transformations tutorial:
        https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html

        :param image gray: Grayscale image containing a potential license plate
        :param int keep: Maximum sorted license plate contours to return
        :return: Candidate license plate contours
        :rtype: List
        """
        # Blackhat morphological operation
        rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
        blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKern)
        if self.debug:
            blackhat_debug = blackhat.copy()
            cv2.putText(blackhat_debug, "Black Hat", (20, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
        
        # Find regions that are light and may contain license plate characters.
        squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        light = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, squareKern)
        light = cv2.threshold(light, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        if self.debug:
            light_debug = light.copy()
            cv2.putText(light_debug, "Light Regions", (20, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

        # Compute the Scharr gradient representation of the blackhat image in the x-direction and then scale the result back to the range [0, 255]
        gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
        gradX = np.absolute(gradX)
        (minVal, maxVal) = (np.min(gradX), np.max(gradX))
        gradX = 255 * ((gradX - minVal) / (maxVal - minVal))
        gradX = gradX.astype("uint8")
        if self.debug:
            scharr_debug = gradX.copy()
            cv2.putText(scharr_debug, "Scharr", (20, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

        # Blur the gradient representation, applying a closing operation, and threshold the image using Otsu's method
        gradX = cv2.GaussianBlur(gradX, (5, 5), 0)
        gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKern)
        thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        if self.debug:
            grad_thresh_debug = thresh.copy()
            cv2.putText(grad_thresh_debug, "GaussianBlur", (20, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

        # Perform a series of erosions and dilations to eliminate noise to the thresholded image
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)
        if self.debug:
            grad_thresh_noise_reduction_debug = thresh.copy()
            cv2.putText(grad_thresh_noise_reduction_debug, "Erode_Dilate", (20, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

        # Take the bitwise AND between the threshold result and the	light regions of the image
        thresh = cv2.bitwise_and(thresh, thresh, mask=light)
        thresh = cv2.dilate(thresh, None, iterations=2)
        thresh = cv2.erode(thresh, None, iterations=1)
        if self.debug:
            bitwise_debug = thresh.copy()
            cv2.putText(bitwise_debug, "Bitwise Dilate Erode", (20, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
            top_debug_image = self.concatenate_horizontally(blackhat_debug, light_debug)
            top_debug_image = self.concatenate_horizontally(top_debug_image, scharr_debug)
            bottom_debug_image = self.concatenate_horizontally(grad_thresh_debug, grad_thresh_noise_reduction_debug)
            bottom_debug_image = self.concatenate_horizontally(bottom_debug_image, bitwise_debug)
            final_debug_image = self.concatenate_vertically(top_debug_image, bottom_debug_image)
            self.debug_imshow("Final Debug Image", final_debug_image, waitKey=True)

        # Find contours in the thresholded image and sort them by their size in descending order, keeping only the largest ones
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:keep]
        return cnts


    def locate_license_plate(self, gray, candidates, clearBorder=False):
        """
        Method that will find the most likely contour containing a license plate out of the set of candidates.

        :param image gray: Input grayscale image
        :param list candidates: License plate candidates returned by locate_license_plate_candidates
        :param bool clearBorder: Whether contours at the edge of the image should be eliminated
        :return: List with license plates' region of interest and contour
        :rtype: List
        """
        # Initialize the license plate contour and Region Of Interest
        licensePlateContour = None
        regionOfInterest = None

        selected_contours = []
        for candidateContour in candidates:
            # Candidate aspect ratio calculation
            (x, y, w, h) = cv2.boundingRect(candidateContour)
            ar = w / float(h)
            # Verify aspect ratio
            if ar >= self.minAspectRatio and ar <= self.maxAspectRatio:
                # Store the license plate contour and extract the license plate from the grayscale image and then threshold it
                licensePlateContour = candidateContour
                licensePlate = gray[y:y + h, x:x + w]
                regionOfInterest = cv2.threshold(licensePlate, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
                
                # Check to see if we should clear any foreground pixels touching the border of the image
                # (which typically, not but always, indicates noise)
                if clearBorder:
                    regionOfInterest = clear_border(regionOfInterest)
                
                # Display any debugging information and then break
                if self.debug:
                    final_debug_image = self.concatenate_horizontally(licensePlate, regionOfInterest)
                    self.debug_imshow("Extracted License Plate", final_debug_image, waitKey=True)
                selected_contours.append((regionOfInterest, licensePlateContour))
        
        return selected_contours

    def build_tesseract_options(self, psm=7):
        """
        Will build the Tessereact config input parameter.

        :param int psm: Page Segmentation Method. There are 13 modes of operation. 
        7 is used by default: "treat the image as a single text line"
        """
        # Only recognize alphanumeric characters
        alphanumeric = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
        options = f"-c tessedit_char_whitelist={alphanumeric}"
        # Set the PSM mode
        options += " --psm {}".format(psm)
        return options

    def find_and_ocr(self, image, psm=7, clearBorder=False):
        """
        Will proceed to find license plates and perform an optical character recognition.

        :param image image: Three-channel color image of car with license plate
        :param int psm: Tesseract Page Segmentation Mode
        :param bool clearBorder: Whether contours at the edge of the image should be eliminated
        :return: License plate detected text and it's contour
        :rtype: 2-tuple (str, contour)
        """
        # Pnitialize the license plate text
        licensePlateText = None
        # Convert the input image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Locate candidate plates
        candidates = self.locate_license_plate_candidates(gray)
        # Locate matching license plate
        selected_contours = self.locate_license_plate(gray, candidates, clearBorder=clearBorder)
        # Only perform the OCR if the license plate ROI is not empty
        text_and_contours = []
        if len(selected_contours) > 0:
            for licensePlate, licensePlateContour in selected_contours:
                # OCR the license plate
                options = self.build_tesseract_options(psm=psm)
                licensePlateText = pytesseract.image_to_string(licensePlate, config=options)
                self.debug_imshow("License Plate", licensePlate)
                text_and_contours.append((licensePlateText, licensePlateContour))
        
        return text_and_contours

Funcion auxiliar para filtrar caracteres que no seas ASCII

In [3]:
def cleanup_text(text):
	# strip out non-ASCII text so we can draw the text on the image
	return "".join([c if ord(c) < 128 else "" for c in text]).strip()

Función auxiliar para calcular como de buena ha sido la detección

In [4]:
def calculate_predicted_accuracy(actual_plate, predicted_plate):
    accuracy = "0 %"
    num_matches = 0
    if actual_plate == predicted_plate:
        accuracy = "100 %"
    else:
        if len(actual_plate) == len(predicted_plate):
            for a, p in zip(actual_plate, predicted_plate):
                if a == p:
                    num_matches += 1
            accuracy = str(round((num_matches / len(actual_plate)), 2) * 100)
            accuracy += "%"
    
    print(f"{actual_plate.ljust(24)}{predicted_plate.ljust(27)}{accuracy}")

## Recorrido y resultados
Se recorre un conjunto de imagenes de matrículas seleccionado. Para cada imagen, si detecta algo, muestra la imagen con el contorno seleccionado que cree que es una matrícula. En caso contrario, no muestra nada. Adicionalmente, en caso de detectar una matrícula, compara ambas ristras y devuelve un porcentaje de los caracteres detectados que están colocados correctamente.

In [7]:
# Path to Tesseract engine
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Instance of the recognition class
#automaticLicensePlateRecognition = AutomaticLicensePlateRecognition(debug=True)
automaticLicensePlateRecognition = AutomaticLicensePlateRecognition()

path_for_license_plates = os.getcwd() + "./license_plates/**/*.jpg"

print(f"Actual License Plate    Predicted License Plate    Accuracy")
print(f"--------------------    -----------------------    --------")

# Iterate over the license plates
for path_to_license_plate in glob.glob(path_for_license_plates, recursive=True):
    # Get license plates
    # Linux
    #license_plate_file = path_to_license_plate.split("/")[-1]
    # Windows
    license_plate_file = path_to_license_plate.split("\\")[-1]
    license_plate, _ = os.path.splitext(license_plate_file)

    # Image read
    img = cv2.imread(path_to_license_plate)
    # Resize with imutils (used to have images that fit in the monitor when debugging)
    img = imutils.resize(img, width=400)

    #text_and_contours = automaticLicensePlateRecognition.find_and_ocr(img)
    # Depending on the clear border (strop foreground pixels that touch border of the image) we can get different results
    text_and_contours = automaticLicensePlateRecognition.find_and_ocr(img, clearBorder=True)
    
    if len(text_and_contours) > 0:
        for licensePlateText, licensePlateContour in text_and_contours:
            # Fit a rotated bounding box to the license plate contour and draw the bounding box on the license plate
            box = cv2.boxPoints(cv2.minAreaRect(licensePlateContour))
            box = box.astype("int")
            cv2.drawContours(img, [box], -1, (0, 255, 0), 2)
            # Add text on the image
            (x, y, w, h) = cv2.boundingRect(licensePlateContour)
            cv2.putText(img, cleanup_text(licensePlateText), (x, y - 15),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
            calculate_predicted_accuracy(license_plate, cleanup_text(licensePlateText))
            cv2.imshow("Output ALPR", img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
    else: 
        calculate_predicted_accuracy(license_plate, "None")



Actual License Plate    Predicted License Plate    Accuracy
--------------------    -----------------------    --------
0802HFP                 None                       0 %
1159FPG                 BISSFP                     0 %
1319FSX                 None                       0 %
2522LNH                 None                       0 %
2711LKN                 None                       0 %
2942HFB                 PS42HFE                    56.99999999999999%
3838KWB                 3838KWB                    100 %
4078BVX                 None                       0 %
4841LFS                 4841LFS                    100 %
4950KZK                 4950K7K                    86.0%
5239LSB                 None                       0 %
5921LMH                 None                       0 %
6298KSN                 None                       0 %
6299JJL                 6299JF                     0 %
7270GVF                 None                       0 %
8168GJG                 8168GJG   